## BRONZE TO SILVER LAYER

### GOLD LAYER - PROCESS HOLDING RECORDS HISTORY


In [6]:
# Import necessary libraries and utility functions
import pandas as pd
from common_utilities import global_path, logger
from datetime import datetime

### Data Processing

# - Load and Filter trade history data from the Gold layer.

# Load trade history from CSV into DataFrame
df = pd.read_csv(global_path.holdings_gold_file_path)
logger.info(
    f"Loaded GOLD Layer trade history data from: {global_path.tradehistory_gold_file_path}"
)

# # Filter for specific segments
# df = df[df["segment"].isin(["EQ", "MF"])]

# Convert 'datetime' column to datetime objects
df["datetime"] = pd.to_datetime(df["datetime"])

# Add a 'date' column by extracting the date part from 'datetime'
df["date"] = df["datetime"].dt.date

# Sort DataFrame by 'segment', 'stock_name', and 'datetime'
df = df.sort_values(by=["segment", "stock_name", "datetime"])


### Data Processing

# - Process data to include all dates up to today.
# - Merge with stock price data and calculate current values.
# - Save the processed data to the Gold layer.

# Get maximum 'datetime' for each 'date' and 'stock_name' combination

max_datetime_df = df.groupby(["date", "stock_name"])["datetime"].max().reset_index()
                
# Retain only rows with maximum datetime for each 'date' and 'stock_name'

df = df.merge(max_datetime_df, on=["date", "stock_name", "datetime"])

# Sort the DataFrame by 'segment', 'stock_name', and 'date'

df = df.sort_values(by=["segment", "stock_name", "date"]).reset_index(drop=True)

# Process each stock name separately

stock_names = df["stock_name"].unique()

result = []

for stock_name in stock_names:
    stock_data = df[df["stock_name"] == stock_name].copy()

    # Set 'date' as index and drop duplicate indices, keeping the first occurrence
    stock_data = stock_data.set_index("date")
    stock_data = stock_data[~stock_data.index.duplicated(keep='first')]

    # Ensure there are no duplicate dates before reindexing
    date_range = pd.date_range(
        start=stock_data.index.min(),
        end=datetime.today().date(),
        freq="D",
    )

    # Reindex to include all dates and forward fill missing values
    stock_data = stock_data.reindex(date_range, method="ffill")

    # Reset index to bring 'date' back as a column
    stock_data = stock_data.reset_index().rename(columns={"index": "date"})
    result.append(stock_data)

# Combine processed DataFrames into one

df = pd.concat(result, ignore_index=True)

# Load stock prices and merge with the main DataFrame

df_StockPrice = pd.read_csv(global_path.stockprice_silver_file_path)

df_StockPrice = df_StockPrice[["date", "stock_name", "close"]]

df_StockPrice["date"] = pd.to_datetime(df_StockPrice["date"])

logger.info(
    f"Loaded SILVER Layer stock price data from: {global_path.stockprice_silver_file_path}"
)

# Merge stock price data

df = pd.merge(df, df_StockPrice, on=["date", "stock_name"], how="left")

# Rename columns for clarity and calculate current value

df = df.rename(
    columns={
        "holding_amount": "investment",
        "holding_quantity": "quantity",
        "close": "ltp",
    }
)

df["current_value"] = df["ltp"] * df["quantity"]

# Calculate PnL and percentage

df["pnl_amount"] = df["current_value"] - df["investment"]

df["pnl_percentage"] = (df["pnl_amount"] / df["investment"]) * 100

# Filter out rows with zero 'holding_quantity'

df = df[(df["investment"] != 0) & (df["current_value"] != 0)]

# Round the values to two decimal places

df = df.round(2)

# Final sorting and column selection

df = df.sort_values(by=["segment", "stock_name", "date"]).reset_index(drop=True)

# Save the final DataFrame to a CSV file

df = df[
    [
        "date",
        "segment",
        "stock_name",
        "quantity",
        "holding_price_avg",
        "investment",
        "ltp",
        "current_value",
        "pnl_amount",
        "pnl_percentage",
    ]
]
df.to_csv(global_path.holdings_gold_file_path, index=None)
logger.info("GOLD Layer CSV file for Holdings successfully created at:")
logger.info(global_path.holdings_gold_file_path.resolve())
# Display DataFrame information and print success message
df.info()


2024-08-05T18:51:22Z - INFO - Loaded GOLD Layer trade history data from: C:\Users\prashant.tripathi\Code\Upstox\DATA\GOLD\TradeHistory\TradeHistory_data.csv


KeyError: 'datetime'

In [ ]:
# Group by segment, exchange, and stock_name
grouped = df.groupby(['segment', 'stock_name'])

# Define a function to get the row with the max datetime in each group
def get_max_datetime_row(group):
    # Find the row with the maximum datetime
    max_datetime_row = group.loc[group['datetime'].idxmax()]
    return max_datetime_row[['holding_quantity', 'holding_price_avg', 'holding_amount']]

# Apply the function to each group
result = grouped.apply(get_max_datetime_row).reset_index()

KeyError: 'datetime'